In [1]:
import copy
import math

# 外皮情報LV3からLV4へのコンバート

## 1. 外皮性能計算プログラム＋外皮用途別情報

- 外皮性能計算プログラムに、外皮の用途別情報を追加したものより、内壁等（建物内部の壁、天井、床）の面積の推定および熱貫流率の決定を行う。
- 外皮の用途別情報が入力された場合において床下空間に属する土間床等面積が0㎡より大きい場合、室と床下空間の間の床の面積の合計は、床下空間に属する土間床等面積の合計（ただし、室用途別の床面積と方位が「下面」の部位面積の差分の合計を上限とする）に等しいとみなす。室と床下空間の間の床は、室用途別の床面積と方位が「下面」の部位面積の差分に応じて、各用途に割り当てる。
- 室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）。
- 内壁等の面積は、室の周長を求める際の係数（アスペクト比）と階高を仮定して、外皮の用途別情報および用途別床面積より推定する。主たる居室―非居室間、その他居室―非居室間は、それぞれ、各用途の床面積が0㎡より大きい場合は、居室と非居室は幅1m、高さ2m以上の通路等で繋がっていると想定する。
- 内壁等の熱貫流率は一般的な仕様を想定して与える（石膏ボード12.5mm－空気層－石膏ボード12.5mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）。室内戸は考慮しない。
- その他は、内壁の用途別情報が得られている場合と同様に求める。

### 1) 内壁等の構成等の設定

#### Outline

- 内壁：せっこうボード12.5mm＋非密閉空気層＋せっこうボード12.5mm
- 内壁床：合板12mm

内壁構成

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup>2 k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| Ro | - | - | 0.090 | 0 |
| せっこうボード | 0.0095 | 0.22 | 0.043 | 830 |
| 非密閉空気層 |  |  | 0.090 | 0 |
| せっこうボード | 0.0095 | 0.22 | 0.043 | 830 |
| Ri | - | - | 0.090 | 0 |

内壁床構成

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup>2 k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| Ro | - | - | 0.150 | 0 |
| 合板 | 0.012 | 0.16 | 0.075 | 720 |
| Ri | - | - | 0.150 | 0 |

#### Function

In [2]:
def get_innerwall( type ) :
    # type is the inner wall type as string, -

    innerwall = {}
    if type == 'innerwall' :
        innerwall ={
            'Name'      : 'Innerwall',
            'Type'      : 'Innerwall',
            'Structure' : 'Wood',
            'Parts'     : {
                'AreaRatio': 1.0,
                'Layers'   : [ 
                    {'Name': 'GPB', 'Cond': 0.22, 'Thick': 0.0125, 'SpecH': 830.0},
                    {'Name': 'GPB', 'Cond': 1/0.09, 'Thick': 1, 'SpecH': 0},
                    {'Name': 'GPB', 'Cond': 0.22, 'Thick': 0.0125, 'SpecH': 830.0}
                 ]
            }
        }
    elif type == 'innerfloor' :
        innerwall ={
            'Name'      : 'Innerfloor',
            'Type'      : 'Innerfloor',
            'Structure' : 'Wood',
            'Direction' : 'Bottom',
            'Parts'     : {
                'AreaRatio': 1.0,
                'Layers'   : [ {'Name': 'PED', 'Cond': 0.16, 'Thick': 0.012, 'SpecH': 720.0} ]
            }
        }
    else :
        raise ValueError
    
    return innerwall

#### Example

In [3]:
get_innerwall('innerwall')

{'Name': 'Innerwall',
 'Parts': {'AreaRatio': 1.0,
  'Layers': [{'Cond': 0.22, 'Name': 'GPB', 'SpecH': 830.0, 'Thick': 0.0125},
   {'Cond': 11.11111111111111, 'Name': 'GPB', 'SpecH': 0, 'Thick': 1},
   {'Cond': 0.22, 'Name': 'GPB', 'SpecH': 830.0, 'Thick': 0.0125}]},
 'Structure': 'Wood',
 'Type': 'Innerwall'}

In [4]:
get_innerwall('innerfloor')

{'Direction': 'Bottom',
 'Name': 'Innerfloor',
 'Parts': {'AreaRatio': 1.0,
  'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
 'Structure': 'Wood',
 'Type': 'Innerfloor'}

### 2) 床下空間上の床面積の推定

#### Outline

$$ \qquad
A_{dif}=max(0,A_m -\sum_{i}^{}A_{of,m,i}) + max(0,A_o-\sum_{i}^{}A_{of,o,i}) + max(0,A_n-\sum_{i}^{}A_{of,n,i})
\qquad
$$
$$ \qquad
A_{b}= min(A_{dif}, \sum_{i}^{}A_{b,i})
\qquad
$$
$$ \qquad
A_{b,m}= A_{b} \times \frac{ max(0,A_m - \sum_{i}^{}A_{of,m,i}) }{ A_{dif} }
\qquad
$$
$$ \qquad
A_{b,o}= A_{b} \times \frac{ max(0,A_o - \sum_{i}^{}A_{of,o,i}) }{ A_{dif} }
\qquad
$$
$$ \qquad
A_{b,n}= A_{b} \times \frac{ max(0,A_n - \sum_{i}^{}A_{of,n,i}) }{ A_{dif} }
\qquad
$$

$A_{b,m}$: 主たる居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{b,o}$: その他の居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{b,n}$: 非居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_b$: 室と床下空間の間の床面積の合計, m<sup>2</sup>

$A_m$: 主たる居室の床面積, m<sup>2</sup>  
$A_o$: その他の居室の床面積, m<sup>2</sup>  
$A_n$: 非居室の床面積, m<sup>2</sup>   
$A_{of,m,i}$: 方位が「下面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,o,i}$: 方位が「下面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,n,i}$: 方位が「下面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{b,i}$: 床下空間に属する土間床等の部位$i$の面積, m<sup>2</sup>

#### Function 1

##### 用途別床面積を取得する関数

In [38]:
def get_area_floor( d ):
    area_floor={
        'main': d['Common']['MainOccupantRoomFloorArea'],
        'other': d['Common']['OtherOccupantRoomFloorArea'],
        'nonliving': d['Common']['TotalFloorArea'] - d['Common']['MainOccupantRoomFloorArea'] - d['Common']['OtherOccupantRoomFloorArea']
    }
    return area_floor

#### Function 2

##### 床下空間上の床面積を推定する関数

In [57]:
def get_area_floor_over_underfloor( d, definition_space ):
    
    # 用途別床面積の取得
    area_floor = get_area_floor( d )
    
    # 下面（direction:Bottom）の壁・窓・ドア面積の取得
    area_bottom = {}
    area_dif= {}
    for x in definition_space['space'] :
        area_bottom[x] = sum(y['area'] for y in d['Walls']+d['Windows']+d['Doors'] if y['space']==x and y['direction']== 'Bottom' )
        area_dif[x]=max(0, area_floor[x]-area_bottom[x])
    area_dif_sum = sum(area_dif[x] for x in definition_space['space'])
    
    # 床下空間に属する土間床等の面積の取得
    area_earthfloor_underfloor= sum(earthfloor['area'] for earthfloor in d['Earthfloors'] if earthfloor['space']=='underfloor' )
    
    #  室と床下空間の間の床面積の推定
    area_floor_over_underfloor_sum = min( area_dif_sum,area_earthfloor_underfloor)
    area_floor_over_underfloor = {}
    for x in definition_space['space'] :
        area_floor_over_underfloor[x] = area_floor_over_underfloor_sum*area_dif[x]/area_dif_sum
        
    return area_floor_over_underfloor

#### Example

In [61]:
definition_space={
    'space':['main', 'other', 'nonliving' ],
    'spaces':['main to other', 'main to nonliving', 'other to nonliving' ],
    'correspondence':{
        'main to other': ['main', 'other' ],
        'main to nonliving': ['main', 'nonliving' ],
        'other to nonliving': ['other', 'nonliving' ] 
    }  
}

d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 30.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'area': 60, 'direction': 'Top', 'name': 'Ceiling', 'space': 'main', 'structure': 'wood', 'type': 'ceiling' },
        { 'area': 15, 'direction': 'Bottom', 'name': 'Floor', 'space': 'other', 'structure': 'wood', 'type': 'floor' },
    ],
    'Windows': [
        { 'area': 30, 'direction': 'SW', 'name': 'WindowSW', 'space': 'main', 'type': 'single' },
        { 'area': 75, 'direction': 'Bottom', 'name': 'WindowBottom', 'space': 'main', 'type': 'single' },
    ],
    'Doors': [
        { 'area': 15, 'direction': 'NW', 'name': 'DoorNW', 'space': 'nonliving' },
        { 'area': 15, 'direction': 'NE', 'name': 'DoorNE', 'space': 'other' }
    ],
    'Earthfloors': [
        { 'area': 40, 'name': 'other', 'space': 'underfloor' },
        { 'area': 20, 'name': 'entrance', 'space': 'nonliving' }
    ]
}

get_area_floor_over_underfloor(d, definition_space)

{'main': 0.0, 'nonliving': 32.0, 'other': 8.0}

### 3) 内壁等面積の推定

#### Outline

$$ \qquad 
A_{iw,m}=0 \quad
(\quad
A_m=0\quad)
\qquad
\\
$$
$$ \qquad 
A_{iw,o}=0 \quad
(\quad
A_o=0\quad)
\qquad
\\
$$
$$ \qquad 
A_{iw,n}=0 \quad
(\quad
A_n=0\quad)
\qquad
\\
$$

$$ \qquad
A_{iw,m}=A_{w,m}-A_{ow,m}=max(0,(2A_m+4ah\sqrt {A_m})-A_{ow,m}) \quad
(\quad
A_m>0\quad)
\qquad
\\
$$
$$ \qquad
A_{iw,o}=A_{w,o}-A_{ow,o}=max(0,(2A_o+4ah\sqrt {A_o})-A_{ow,o}) \quad
(\quad
A_o>0\quad)
\qquad
\\
$$$$ \qquad
A_{iw,n}=A_{w,n}-A_{ow,n}=max(0,(2A_n+4ah\sqrt {A_n})-A_{ow,n}) \quad
(\quad
A_n>0\quad)
\qquad
\\
$$

$$ \qquad
A_{iw,m} =A_{iw,m-o}+A_{iw,m-n}\quad, \quad
A_{iw,o} =A_{iw,m-o}+A_{iw,o-n}\quad, \quad
A_{iw,n} =A_{iw,m-o}+A_{iw,o-n}\quad
\qquad
\\
$$　より

$$ \qquad 
A_{iw,m-o}=0 \quad
(\quad
A_m=0\quad or \quad
A_o=0\quad)
\qquad
\\
$$
$$ \qquad 
A_{iw,m-n}=0 \quad
(\quad
A_m=0\quad or \quad
A_n=0\quad)
\qquad
\\
$$
$$ \qquad 
A_{iw,o-n}=0 \quad
(\quad
A_o=0\quad or \quad
A_n=0\quad)
\qquad
\\
$$

$$ \qquad
A_{iw,m-o}=max(0,\frac{A_{iw,m}+A_{iw,o}-A_{iw,n}}{2}) \quad
(\quad
A_m>0\quad and \quad
A_o>0\quad)
\qquad
\\
$$
$$ \qquad 
A_{iw,m-n}=max(2,\frac{A_{iw,m}-A_{iw,o}+A_{iw,n}}{2}) \quad
(\quad
A_m>0\quad and \quad
A_n>0\quad)
\qquad
\\
$$
$$ \qquad 
A_{iw,o-n}=max(2,\frac{-A_{iw,m}+A_{iw,o}+A_{iw,n}}{2}) \quad
(\quad
A_o>0\quad and \quad
A_n>0\quad)
\qquad
\\
$$

$A_{w,m}$: 主たる居室の外皮と、主たる居室と床下空間の間の床と、主たる居室の内壁等の面積の合計, m<sup>2</sup>  
$A_{w,o}$: その他の居室の外皮と、その他の居室と床下空間の間の床と、その他の居室の内壁等の面積の合計, m<sup>2</sup>  
$A_{w,n}$: 非居室の外皮と、非居室と床下空間の間の床と、非居室の内壁等の面積の合計, m<sup>2</sup>  
$A_{iw,m}$: 主たる居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,o}$: その他の居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,n}$: 非居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,m-o}$: 主たる居室とその他の居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,m-n}$: 主たる居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,o-n}$: その他の居室と非居室の間の内壁等の面積, m<sup>2</sup>  

$A_{ow,m}$: 主たる居室の外皮と、主たる居室と床下空間の間の床の面積の合計, m<sup>2</sup>  
$A_{ow,o}$: その他の居室の外皮と、その他の居室と床下空間の間の床の面積の合計, m<sup>2</sup>  
$A_{ow,n}$: 非居室の外皮と、非居室と床下空間の間の床の面積の合計, m<sup>2</sup>  
$ar_m$: 主たる居室の周長を求める際の係数, -  
$ar_o$: その他の居室の周長を求める際の係数, -  
$ar_n$: 非居室の周長を求める際の係数, -  
$h$: 階高, m

#### Function

##### 内壁等の面積を推定する関数

In [10]:
def get_area_inner_wall( d, h, ar, definition_space, area_inner_wall_min ):
    # h is the floor height as double, m
    # ar is the aspect ratio of the space as double, -
    # area_min is the minimum area between 2 spaces as float, m2 
    
    # 用途別床面積の取得
    area_floor=get_area_floor( d )
 
    # 外皮と、床下空間の間の床の面積の合計の取得  
    # 室用途別の内壁等の面積の推
    d_ow=[]
    for y in ['Walls','Windows','Doors','InnerWalls'] :
        if (y in d) == True :
            d_ow=d_ow+d[y]  
            
    area_inner_wall_space={}            
    for x in definition_space['space'] :
        if area_floor[x]==0 :
            area_inner_wall_space[x]=0           
        else :
            area_ow = sum(y['area'] for y in d_ow if y['space']== x )
            area_inner_wall_space[x]=max(0,2*area_floor[x]+4*h*ar[x]*area_floor[x]**0.5-area_ow)
    
    # 室用途間の内壁等の面積の推定
    # 室用途床面積が0㎡の場合に、室間の内壁等の面積を0㎡とする
    area_inner_wall={}
    for x in definition_space['spaces'] :        
        if min(area_floor[y] for y in definition_space['correspondence'][x])==0 :
            area_inner_wall[x]=0
        else :
            area_inner_wall[x]=max(area_inner_wall_min[x],
                                    (2*sum(area_inner_wall_space[y] for y in definition_space['correspondence'][x])
                                    - sum(area_inner_wall_space[y] for y in definition_space['space']) )/2)
    
    return area_inner_wall

#### Example

In [11]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 60.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'area': 60, 'direction': 'Top', 'name': 'Ceiling', 'space': 'main', 'structure': 'wood', 'type': 'ceiling' },
        { 'area': 15, 'direction': 'Bottom', 'name': 'Floor', 'space': 'other', 'structure': 'wood', 'type': 'floor' },
    ],
    'Windows': [
        { 'area': 30, 'direction': 'SW', 'name': 'WindowSW', 'space': 'main', 'type': 'single' },
        { 'area': 75, 'direction': 'Bottom', 'name': 'WindowBottom', 'space': 'other', 'type': 'single' },
    ],
    'Doors': [
        { 'area': 15, 'direction': 'NW', 'name': 'DoorNW', 'space': 'nonliving' },
        { 'area': 15, 'direction': 'NE', 'name': 'DoorNE', 'space': 'other' }
    ],
    'Earthfloors': [
        { 'area': 40, 'name': 'other', 'space': 'underfloor' },
        { 'area': 20, 'name': 'entrance', 'space': 'nonliving' }
    ]
}

# 階高, m
h=2.8

# 周長を求める際の係数, -
ar={'main':1.2, 'other':1.4, 'nonliving':1.4 }

# 室間の内壁等の面積の最小値, m2
area_inner_wall_min={ 'main to other':0.0, 'main to nonliving':2.0, 'other to nonliving':2.0 }

# 用途と複数用途間の対応関係の定義
definition_space={
    'space':['main', 'other', 'nonliving' ],
    'spaces':['main to other', 'main to nonliving', 'other to nonliving' ],
    'correspondence':{
        'main to other': ['main', 'other' ],
        'main to nonliving': ['main', 'nonliving' ],
        'other to nonliving': ['other', 'nonliving' ] 
    }  
}

get_area_inner_wall( d, h, ar, definition_space, area_inner_wall_min )

{'main to nonliving': 0,
 'main to other': 135.28127504155998,
 'other to nonliving': 0}

## 2. Convert

### 1) 共通

'Common'要素はそのまま維持する。

参照：InputDataDefinition.ipynb

#### Output

参照：InputDataDefinition.ipynb

#### Function

In [12]:
def convert_common(d):
    return d['Common']

#### Example

In [13]:
d = {
    'Common': {
        'Region' : 6,
        'IsSimplifiedInput': True,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    }
}

convert_common(d)

{'IsSimplifiedInput': True,
 'MainOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloorArea': 60.0,
 'Region': 6,
 'TotalFloorArea': 120.0}

### 2) 開口部以外の不透明部位

'Walls'要素はそのまま維持する。

#### Function

In [14]:
def convert_wall(d):
    return d['Walls']

#### Example

In [15]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Walls': [{'area': 16.95, 'direction': 'top', 'name': 'Ceiling_main', 'space': 'main'},
        {'area': 33.9, 'direction': 'top', 'name': 'Ceiling_other', 'space': 'other'},
        {'area': 16.95, 'direction': 'top', 'name': 'Ceiling_nonliving', 'space': 'nonliving'},
        {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_main', 'space': 'main'},
        {'area': 20.315, 'direction': 'SW', 'name': 'Wall_SW_other', 'space': 'other'},
        {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_nonliving', 'space': 'nonliving'}
    ]
}    

convert_wall(d)

[{'area': 16.95, 'direction': 'top', 'name': 'Ceiling_main', 'space': 'main'},
 {'area': 33.9, 'direction': 'top', 'name': 'Ceiling_other', 'space': 'other'},
 {'area': 16.95,
  'direction': 'top',
  'name': 'Ceiling_nonliving',
  'space': 'nonliving'},
 {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_main', 'space': 'main'},
 {'area': 20.315,
  'direction': 'SW',
  'name': 'Wall_SW_other',
  'space': 'other'},
 {'area': 10.1575,
  'direction': 'SW',
  'name': 'Wall_SW_nonliving',
  'space': 'nonliving'}]

### 3) 開口部の透明部位

'Windows'要素はそのまま維持する。

#### Function

In [16]:
def convert_window(d):
    return d['Windows']

#### Example

In [17]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Windows': [{'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_main', 'space': 'main'},
        {'area': 15.125, 'direction': 'SW', 'name': 'WindowSW_other', 'space': 'other'},
        {'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_nonliving', 'space': 'nonliving'},
        {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_main', 'space': 'main'},
        {'area': 1.585, 'direction': 'NW', 'name': 'WindowNW_other', 'space': 'other'},
        {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_nonliving', 'space': 'nonliving'}]
}    

convert_window(d)

[{'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_main', 'space': 'main'},
 {'area': 15.125,
  'direction': 'SW',
  'name': 'WindowSW_other',
  'space': 'other'},
 {'area': 7.5625,
  'direction': 'SW',
  'name': 'WindowSW_nonliving',
  'space': 'nonliving'},
 {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_main', 'space': 'main'},
 {'area': 1.585,
  'direction': 'NW',
  'name': 'WindowNW_other',
  'space': 'other'},
 {'area': 0.7925,
  'direction': 'NW',
  'name': 'WindowNW_nonliving',
  'space': 'nonliving'}]

### 4) 開口部の不透明部位

'Doors'要素はそのまま維持する。

#### Function

In [18]:
def convert_door(d):
    return d['Doors']

#### Example

In [19]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Doors': [{'area': 13.0, 'direction': 'NW', 'name': 'DoorNW_main', 'space': 'main'},
        {'area': 26.0, 'direction': 'NW', 'name': 'DoorNW_other', 'space': 'other'},
        {'area': 13.0, 'direction': 'NW', 'name': 'DoorNW_nonliving', 'space': 'nonliving'}]
}    
convert_door(d)

[{'area': 13.0, 'direction': 'NW', 'name': 'DoorNW_main', 'space': 'main'},
 {'area': 26.0, 'direction': 'NW', 'name': 'DoorNW_other', 'space': 'other'},
 {'area': 13.0,
  'direction': 'NW',
  'name': 'DoorNW_nonliving',
  'space': 'nonliving'}]

### 5) 線熱橋

'Heatbridges'要素はそのまま維持する。

#### Function

In [20]:
def convert_heatbridge(d):
    return d['Heatbridges']

#### Example

In [21]:
d = {
    'Common': {
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0,
    },
    'Heatbridges': [
        {'direction1': 'NW', 'direction2': 'NE', 'length': 2.5, 'space': 'main', 'structure': 'rc'},
        {'direction1': 'NW', 'direction2': 'NE', 'length': 5.0, 'space': 'other', 'structure': 'rc'},
        {'direction1': 'NW', 'direction2': 'NE', 'length': 2.5, 'space': 'nonliving', 'structure': 'rc'}
    ]
}    
convert_heatbridge(d)

[{'direction1': 'NW',
  'direction2': 'NE',
  'length': 2.5,
  'space': 'main',
  'structure': 'rc'},
 {'direction1': 'NW',
  'direction2': 'NE',
  'length': 5.0,
  'space': 'other',
  'structure': 'rc'},
 {'direction1': 'NW',
  'direction2': 'NE',
  'length': 2.5,
  'space': 'nonliving',
  'structure': 'rc'}]

### 6) 土間床等の外周部の線熱橋

'EarthfloorPerimeters'要素はそのまま維持する。

#### Function

In [22]:
def convert_earthfloorperimeter(d):
    return d['EarthfloorPerimeters']

#### Example

In [23]:
d = {
    'EarthfloorPerimeters': [
        {'direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8, 'space': 'underfloor'},
        {'direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8, 'space': 'underfloor'}]
}    
convert_earthfloorperimeter(d)

[{'direction': 'NW',
  'length': 2.43,
  'name': 'other_NW',
  'psi': 1.8,
  'space': 'underfloor'},
 {'direction': 'NE',
  'length': 2.43,
  'name': 'other_NE',
  'psi': 1.8,
  'space': 'underfloor'}]

### 7) 土間床

'Earthfloors'要素はそのまま維持する。

#### Function

In [24]:
def convert_earthfloor(d):
    return d['Earthfloors']

#### Example

In [25]:
d = {
    'Earthfloors': [{'area': 5.0, 'name': 'other', 'space': 'underfloor'}]
}    
convert_earthfloor(d)

[{'area': 5.0, 'name': 'other', 'space': 'underfloor'}]

### 8) 内壁等

室と床下空間の間の床、内壁等の推定を行う。

#### Function 1

##### 室と床下空間の間の床を推定する関数

In [26]:
def get_innerwall_floor( d, innerwalls, definition_space ):
    
    # 室と床下空間の間の床面積の推定
    innerfloor_temp = get_innerwall( 'innerfloor' )
    areas_innerfloor = get_area_floor_over_underfloor(d, definition_space)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    for space in definition_space['space'] :
        innerfloor = copy.deepcopy(innerfloor_temp)
        innerfloor['Name'] = innerfloor_temp['Name'] + '_' + space 
        innerfloor['Area'] = areas_innerfloor[space]
        innerfloor['Space'] = space
        innerfloor['NextSpace'] = 'Underfloor'
        innerwalls.append(innerfloor)
 
        innerfloor_underfloor = copy.deepcopy(innerfloor)
        innerfloor_underfloor['Space'] = 'Underfloor'
        innerfloor_underfloor['NextSpace'] = space
        innerwalls.append(innerfloor_underfloor)
        
        return innerwalls

#### Function 2

##### 内壁を推定する関数

In [27]:
def get_innerwall_wall( d, innerwalls, h, ar, definition_space, area_inner_wall_min ):

    # 内壁面積の推定
    innerwall_temp = get_innerwall( 'innerwall' )
    areas_innerwall = get_area_inner_wall( d, h, ar, definition_space, area_inner_wall_min )
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    for spaces in definition_space['spaces'] :
        innerwall = copy.deepcopy(innerwall_temp)
        innerwall['Name'] = innerwall_temp['Name'] + '_' + spaces 
        innerwall['Area'] = areas_innerwall[spaces]
        innerwall['Space'] = definition_space['correspondence'][spaces][0]
        innerwall['NextSpace'] = definition_space['correspondence'][spaces][1]
        innerwalls.append(innerwall)

        innerwall_next = copy.deepcopy(innerwall)
        innerwall_next['Space'] = definition_space['correspondence'][spaces][1]
        innerwall_next['NextSpace'] = definition_space['correspondence'][spaces][0]
        innerwalls.append(innerwall_next)

        return innerwalls

#### Function 3

##### 内壁等のコンバート関数

In [28]:
def convert_innerwall( d ):

    # 用途と複数用途間の対応関係の定義
    definition_space={
        'space':['main', 'other', 'nonliving' ],
        'spaces':['main to other', 'main to nonliving', 'other to nonliving' ],
        'correspondence':{
            'main to other': ['main', 'other' ],
            'main to nonliving': ['main', 'nonliving' ],
            'other to nonliving': ['other', 'nonliving' ] 
        }  
    }
    
    # 階高, m
    h=2.8

    # 周長を求める際の係数, -
    ar={'main':1.2, 'other':1.4, 'nonliving':1.4 }

    # 室間の内壁等の面積の最小値, m2
    area_inner_wall_min={'main to other':0.0, 'main to nonliving':2.0, 'other to nonliving':2.0 }
    
    innerwalls =[]
    
    # 室と床下空間の間の床の推定
    innerwalls= get_innerwall_floor( d, innerwalls, definition_space )
    
    # 内壁等の推定              
    innerwalls= get_innerwall_wall( d, innerwalls, h, ar, definition_space, area_inner_wall_min )
            
    return innerwalls

#### Example

In [29]:
d = {
    'Common': {'MainOccupantRoomFloorArea': 30.0, 'OtherOccupantRoomFloorArea': 60.0, 'Region': 6, 'TotalFloorArea': 120.0},
    'Doors': [
        {'area': 0.63, 'direction': 'NW', 'name': 'DoorNW_main', 'space': 'main'},
        {'area': 1.26, 'direction': 'NW', 'name': 'DoorNW_other', 'space': 'other'},
        {'area': 0.63, 'direction': 'NW', 'name': 'DoorNW_nonliving', 'space': 'nonliving'},
        {'area': 0.54, 'direction': 'NE', 'name': 'DoorNE_main', 'space': 'main'},
        {'area': 1.08, 'direction': 'NE', 'name': 'DoorNE_other', 'space': 'other'},
        {'area': 0.54, 'direction': 'NE', 'name': 'DoorNE_nonliving', 'space': 'nonliving'}
     ],
    'EarthfloorPerimeters': [
        {'direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8, 'space': 'underfloor'},
        {'direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8, 'space': 'underfloor'}
    ],
    'Earthfloors': [
        {'area': 5.0, 'name': 'other', 'space': 'underfloor'},
        {'area': 5.0, 'name': 'entrance', 'space': 'underfloor'}
    ],
    'Walls': [
        {'area': 16.95, 'direction': 'top', 'name': 'Ceiling_main', 'space': 'main', 'structure': 'wood', 'type': 'ceiling'},
        {'area': 33.9, 'direction': 'top', 'name': 'Ceiling_other', 'space': 'other', 'structure': 'wood', 'type': 'ceiling'},
        {'area': 16.95, 'direction': 'top', 'name': 'Ceiling_nonliving', 'space': 'nonliving', 'structure': 'wood', 'type': 'ceiling'},
        {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_main', 'space': 'main', 'structure': 'wood', 'type': 'wall'},
        {'area': 20.315, 'direction': 'SW', 'name': 'Wall_SW_other', 'space': 'other', 'structure': 'wood', 'type': 'wall'},
        {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_nonliving', 'space': 'nonliving', 'structure': 'wood', 'type': 'wall'}
    ],
    'Windows': [
        {'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_main', 'space': 'main', 'type': 'single'},
        {'area': 15.125, 'direction': 'SW', 'name': 'WindowSW_other', 'space': 'other', 'type': 'single'},
        {'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_nonliving',  'space': 'nonliving', 'type': 'single'},
         {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_main', 'space': 'main', 'type': 'single'},
        {'area': 1.585, 'direction': 'NW', 'name': 'WindowNW_other', 'space': 'other', 'type': 'single'},
        {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_nonliving', 'space': 'nonliving', 'type': 'single'}
        ]
}

convert_innerwall( d )

[{'Area': 2.5,
  'Direction': 'Bottom',
  'Name': 'Innerfloor_main',
  'NextSpace': 'Underfloor',
  'Parts': {'AreaRatio': 1.0,
   'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
  'Space': 'main',
  'Structure': 'Wood',
  'Type': 'Innerfloor'},
 {'Area': 2.5,
  'Direction': 'Bottom',
  'Name': 'Innerfloor_main',
  'NextSpace': 'main',
  'Parts': {'AreaRatio': 1.0,
   'Layers': [{'Cond': 0.16, 'Name': 'PED', 'SpecH': 720.0, 'Thick': 0.012}]},
  'Space': 'Underfloor',
  'Structure': 'Wood',
  'Type': 'Innerfloor'},
 {'Area': 77.96138622447441,
  'Name': 'Innerwall_main to other',
  'NextSpace': 'other',
  'Parts': {'AreaRatio': 1.0,
   'Layers': [{'Cond': 0.22, 'Name': 'GPB', 'SpecH': 830.0, 'Thick': 0.0125},
    {'Cond': 11.11111111111111, 'Name': 'GPB', 'SpecH': 0, 'Thick': 1},
    {'Cond': 0.22, 'Name': 'GPB', 'SpecH': 830.0, 'Thick': 0.0125}]},
  'Space': 'main',
  'Structure': 'Wood',
  'Type': 'Innerwall'},
 {'Area': 77.96138622447441,
  'Name': 'Innerw

## 3. Lv4入力情報のコンバート（統合）
- 1)共通情報、2)開口部以外の不透明部位、3)開口部の透明部位、4)開口部の不透明部位、5) 線熱橋、6)土間床等の外周部の線熱橋、7)土間床、8)内壁等の各コンバート関数を統合し、Lv3の入力情報をLv4の入力情報にコンバートする。

### Function

In [30]:
def convert(d):
    d_lv4_input = {}
    d_lv4_input['Common'] = convert_common(d)
    if ('Walls' in d) == True:
        d_lv4_input['Walls'] = convert_wall(d)
    if ('Windows' in d) == True:
        d_lv4_input['Windows'] = convert_window(d)
    if ('Doors' in d) == True:
        d_lv4_input['Doors'] = convert_door(d)
    if ('EarthfloorPerimeters' in d) == True:
        d_lv4_input['EarthfloorPerimeters'] = convert_earthfloorperimeter(d)
    if ('Earthfloors' in d) == True:
        d_lv4_input['Earthfloors'] = convert_earthfloor(d)
    if ('Heatbridges' in d) == True:
        d_lv4_input['Heatbridges'] = convert_heatbridge(d)
    d_lv4_input['InnerWalls'] = convert_innerwall(d)

    return d_lv4_input

### Example

In [31]:
d = {
    'Common': {'MainOccupantRoomFloorArea': 30.0, 'OtherOccupantRoomFloorArea': 60.0, 'Region': 6, 'TotalFloorArea': 120.0},
    'Doors': [
        {'area': 0.63, 'direction': 'NW', 'name': 'DoorNW_main', 'space': 'main'},
        {'area': 1.26, 'direction': 'NW', 'name': 'DoorNW_other', 'space': 'other'},
        {'area': 0.63, 'direction': 'NW', 'name': 'DoorNW_nonliving', 'space': 'nonliving'},
        {'area': 0.54, 'direction': 'NE', 'name': 'DoorNE_main', 'space': 'main'},
        {'area': 1.08, 'direction': 'NE', 'name': 'DoorNE_other', 'space': 'other'},
        {'area': 0.54, 'direction': 'NE', 'name': 'DoorNE_nonliving', 'space': 'nonliving'}
     ],
    'EarthfloorPerimeters': [
        {'direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8, 'space': 'underfloor'},
        {'direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8, 'space': 'underfloor'}
    ],
    'Earthfloors': [
        {'area': 5.0, 'name': 'other', 'space': 'underfloor'},
        {'area': 5.0, 'name': 'entrance', 'space': 'underfloor'}
    ],
    'Walls': [
        {'area': 16.95, 'direction': 'top', 'name': 'Ceiling_main', 'space': 'main', 'structure': 'wood', 'type': 'ceiling'},
        {'area': 33.9, 'direction': 'top', 'name': 'Ceiling_other', 'space': 'other', 'structure': 'wood', 'type': 'ceiling'},
        {'area': 16.95, 'direction': 'top', 'name': 'Ceiling_nonliving', 'space': 'nonliving', 'structure': 'wood', 'type': 'ceiling'},
        {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_main', 'space': 'main', 'structure': 'wood', 'type': 'wall'},
        {'area': 20.315, 'direction': 'SW', 'name': 'Wall_SW_other', 'space': 'other', 'structure': 'wood', 'type': 'wall'},
        {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_nonliving', 'space': 'nonliving', 'structure': 'wood', 'type': 'wall'}
    ],
    'Windows': [
        {'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_main', 'space': 'main', 'type': 'single'},
        {'area': 15.125, 'direction': 'SW', 'name': 'WindowSW_other', 'space': 'other', 'type': 'single'},
        {'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_nonliving',  'space': 'nonliving', 'type': 'single'},
         {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_main', 'space': 'main', 'type': 'single'},
        {'area': 1.585, 'direction': 'NW', 'name': 'WindowNW_other', 'space': 'other', 'type': 'single'},
        {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_nonliving', 'space': 'nonliving', 'type': 'single'}
        ]
}

convert(d)

{'Common': {'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'Region': 6,
  'TotalFloorArea': 120.0},
 'Doors': [{'area': 0.63,
   'direction': 'NW',
   'name': 'DoorNW_main',
   'space': 'main'},
  {'area': 1.26, 'direction': 'NW', 'name': 'DoorNW_other', 'space': 'other'},
  {'area': 0.63,
   'direction': 'NW',
   'name': 'DoorNW_nonliving',
   'space': 'nonliving'},
  {'area': 0.54, 'direction': 'NE', 'name': 'DoorNE_main', 'space': 'main'},
  {'area': 1.08, 'direction': 'NE', 'name': 'DoorNE_other', 'space': 'other'},
  {'area': 0.54,
   'direction': 'NE',
   'name': 'DoorNE_nonliving',
   'space': 'nonliving'}],
 'EarthfloorPerimeters': [{'direction': 'NW',
   'length': 2.43,
   'name': 'other_NW',
   'psi': 1.8,
   'space': 'underfloor'},
  {'direction': 'NE',
   'length': 2.43,
   'name': 'other_NE',
   'psi': 1.8,
   'space': 'underfloor'}],
 'Earthfloors': [{'area': 5.0, 'name': 'other', 'space': 'underfloor'},
  {'area': 5.0, 'name': 'entrance', 'spa